# Imports

In [2]:
from fastai.vision.all import *
from fastai.vision.core import *
from fastai.callback.fp16 import *

from fastai.callback.cutmix import *
from torch.distributions.beta import Beta

from fastai.callback.wandb import *
import torchvision.models as models
import pandas as pd
import numpy as np
import distillation

#from efficientnet_pytorch import EfficientNet
import albumentations
import wandb
#import sys
#sys.path.append('../input/timm2021/pytorch-image-models-master')
#import timm

# Setup

In [3]:
class Config:
    testing     = False # must be same as create-folds.ipynb
    image_size  = 512
    batch_size  = 16
    epochs      = 10
    f_epochs    = 1
    train_folds = ['f2']
    arch        = 'efficientnet-b4'
    
cfg = Config()

In [4]:
def set_seeds():
    random.seed(42)
    np.random.seed(42)
    torch.manual_seed(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
set_seeds()

In [5]:
path_str = '../input/cassava-leaf-disease-merged'

images_path = Path(path_str + '/train')
csv_path = Path(path_str + '/merged.csv')
folds_path = Path('../input/fold-indexes/folds-merged.csv')

full_df = pd.read_csv(csv_path)
folds_df = pd.read_csv(folds_path)

# drop rows so we get an even number for our folds and remove duplicates
full_df = full_df[~full_df['image_id'].isin(['1562043567.jpg', '3551135685.jpg', '2252529694.jpg', '1000015157.jpg', '1000201771.jpg', '100042118.jpg', '1001723730.jpg'])]

# Create a test dataset

In [6]:
if cfg.testing:
    full_df = full_df.tail(120)#.head(120)
else:
    wandb.login(key="11b470b697ff94b3896d2243b147d42177a5cb7a")
    wandb.init(project="cassava", entity="teo03")

len(full_df)

wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: teo03 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.19 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


26330

# Augmentation and train functions

In [7]:
class AlbumentationsTransform(RandTransform):
    split_idx,order = None, 2
    
    def __init__(self, train_aug, valid_aug): 
        store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)


def get_train_aug(size): 
    return albumentations.Compose([
            albumentations.RandomResizedCrop(size,size),
            albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
            albumentations.CoarseDropout(p=0.5),
            albumentations.Cutout(p=0.5)
])

def get_valid_aug(size): 
    return albumentations.Compose([
        albumentations.Resize(size, size),
        albumentations.CenterCrop(size, size, p=1.),
], p=1.)

def get_x(row): return images_path/row['image_id']
def get_y(row): return row['label']

In [8]:
def train(dls, fold):
    
    #model = EfficientNet.from_pretrained(cfg.arch, num_classes=5)
    model = models.resnext50_32x4d(pretrained=True)
    n_features = model.fc.in_features
    model.fc = nn.Linear(n_features, 5)
    
    fold_name = f'model-{fold}'

    # define learners
    t_learn = Learner(
        dls=dls,
        model=model,
        opt_func=ranger,
        metrics=accuracy,
        loss_func=LabelSmoothingCrossEntropy(),
        cbs=[CutMix()]
    ).to_fp16()
    
    s_learn = Learner(
        dls=dls,
        model = model,
        opt_func=ranger,
        metrics=accuracy,
        loss_func=LabelSmoothingCrossEntropy(),
        cbs=[
            WandbCallback(log_preds=False, log_model=True, n_preds=2),
            SaveModelCallback(
                monitor='accuracy',
                fname=fold_name,
                with_opt=True,
                every_epoch=False
            ),
            CutMix(),
        ]
    ).to_fp16()
    
    lr = 0.001
    
    # teacher model training
    if not cfg.testing:
        lr_min, lr_steep = t_learn.lr_find(show_plot=False)
        lr = round(lr_min, 5)
        print(f'found lr of({lr_min}): {round(lr_min, 5)}')
    

    t_learn.fine_tune(
        cfg.epochs,
        base_lr=lr,
        freeze_epochs=cfg.f_epochs,
    )
    
    
    # student model training
    if not cfg.testing:
        lr_min, lr_steep = s_learn.lr_find(show_plot=False)
        lr = round(lr_min, 5)
        print(f'found lr of({lr_min}): {round(lr_min,5)}')
        
    s_learn.fine_tune(
        cfg.epochs,
        base_lr=lr,
        freeze_epochs=cfg.f_epochs,
        cbs = [distillation.KnowledgeDistillation(s_learn,t_learn)]
    )
    
    s_learn.load(fold_name) # load the best .pth
    s_learn.export(fold_name + '.pkl') # export as .pkl
    
    return s_learn

# Training
train models on different folds of our data

In [9]:
for fold in cfg.train_folds:
    val_index = folds_df[fold].to_numpy()
    
    print(f'started training on {fold}')
    
    train_block = DataBlock(
        blocks=(ImageBlock, CategoryBlock),
        get_x=get_x,
        get_y=get_y,
        splitter=IndexSplitter(val_index),
        item_tfms= [
            AlbumentationsTransform(
                get_train_aug(size=cfg.image_size),
                get_valid_aug(size=cfg.image_size)
            )
        ],
        batch_tfms=[Normalize.from_stats(*imagenet_stats)]
    )

    dls = train_block.dataloaders(full_df, bs=cfg.batch_size)
    learn = train(dls, fold)

print(f'training on {cfg.train_folds} done')

started training on f2


Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth


  0%|          | 0.00/95.8M [00:00<?, ?B/s]

found lr of(0.00036307806149125097): 0.00036


epoch,train_loss,valid_loss,accuracy,time
0,0.984100,0.744054,0.835169,15:52


...

found lr of(8.31763736641733e-07): 0.0


epoch,train_loss,valid_loss,accuracy,time
0,0.828110,0.617928,0.897455,15:48


Better model found at epoch 0 with accuracy value: 0.8974553942680359.


epoch,train_loss,valid_loss,accuracy,time
0,0.845611,0.616480,0.898025,15:56
1,0.808787,0.615922,0.897645,15:56
2,0.803948,0.613997,0.898025,15:58
3,0.833842,0.618190,0.895556,15:55
4,0.827288,0.615982,0.897265,15:54
5,0.819964,0.616442,0.897645,15:54
6,0.833721,0.614693,0.897645,15:57
7,0.812893,0.614553,0.898405,15:57
8,0.805492,0.616613,0.897455,15:58
9,0.794504,0.614755,0.897835,15:57


Better model found at epoch 0 with accuracy value: 0.8980250954627991.
Better model found at epoch 7 with accuracy value: 0.8984048366546631.
training on ['f2'] done
